### Generar resultado "results_group_movie_genre"

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
file_date = dbutils.widgets.get("file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Obtener datos de Silver

In [0]:
df_movies = spark.read.format("delta").load(f"{silver_folder_path}/movies").filter(f"file_date = '{file_date}'")
df_movie_genres = spark.read.format("delta").load(f"{silver_folder_path}/movie_genres").filter(f"file_date = '{file_date}'")
df_genres = spark.read.format("delta").load(f"{silver_folder_path}/genres")

In [0]:
df_movie_genre = df_genres.join(df_movie_genres,df_genres.genre_id==df_movie_genres.genre_id,"inner").select("movie_id","genre_name")

In [0]:
df_movie_full = df_movies.join(df_movie_genre,df_movies.movie_id==df_movie_genre.movie_id,"inner").select("year_release_date","budget","revenue","genre_name")

##### Agregaciones

In [0]:
from pyspark.sql.functions import sum

results_group_by = df_movie_full.groupBy("year_release_date","genre_name") \
                                .agg(
                                    sum("budget").alias("total_budget"),
                                    sum("revenue").alias("total_revenue")
                                    )

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,dense_rank,asc,lit

df_results_dense_rank = Window.partitionBy("year_release_date").orderBy(desc("total_budget"),desc("total_revenue"))
df_final = results_group_by.withColumn("dense_rank",dense_rank().over(df_results_dense_rank)).withColumn("created_date",lit(file_date))

##### Escribir resultado en Gold

In [0]:
merge_condition="target.year_release_date = source.year_release_date  AND target.genre_name = source.genre_name AND target.created_date = source.created_date"
merge_delta_lake(df_final,"movie_gold","results_group_movie_genre",gold_folder_path,merge_condition,"created_date")

In [0]:
%sql
select * from movie_gold.results_group_movie_genre